# This script :
1. imports files from a Google Drive folder;
2. extracts text data and file name; and
3. stores data for later use

The script's text extraction process scans each PDF file with pdfplumber. If pdfplumber doesn't work (if the length of the extracted text is under 200 words), we also scan the file with an OCR engine (easyocr, which uses the GPU; or pytesseract, which doesn't use the GPU).

The script utilizes a metastore.json file to track progress, so if your Colab shuts down mid-execution, you can resume text extraction from where you left off.

The script outputs a csv file (extracted_text.csv) that stores the data in 3 columns: filename, text (extracted with pdfplumber), and ocr_text (extracted with easyocr).

In [ ]:
!pip install pdfplumber pytesseract opencv-python-headless easyocr
!sudo apt install tesseract-ocr
!pip install pdf2image
!apt-get install -y poppler-utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 22.6 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 

In [ ]:
# PDF parsing imports
import pdfplumber
import easyocr # leverages GPU, so it's faster
import pytesseract # does not leverage GPU (?), so it's slow

# general imports
import pandas as pd
import numpy as np
import io
import re
import tempfile
import os

from pdf2image import convert_from_path
from PIL import Image

from tqdm import tqdm  # For progress bar
import json #for saving metadata
import torch

### Functions to import and process file

In [ ]:
# function to extract text from pdf using pdfplumber
def extract_text_with_pdfplumber(pdf_path):
  try:
    with pdfplumber.open(pdf_path) as pdf:
      text = ''
      for page in pdf.pages:
        text += page.extract_text() or '' # handle None returns
      return text
  except Exception as e:
    print(f'pdfplumber error {e}')
    return ''

# function to extract text from pdf using OCR (easyocr)
def extract_text_with_ocr(pdf_path):
  try:
      images = convert_from_path(pdf_path, dpi=75)  # adjust DPI if needed
      text = ''
      for image in images:
          result = reader.readtext(np.array(image)) # convert image to numpy array for easyocr ingestion
          extracted_text = ' '.join([detection[1] for detection in result])
          text += extracted_text + ' '  # add space between lines/detections
      return text
  except Exception as e:
      print(f'EasyOCR error: {e}')
      return ''

# function to load metadata (will help track progress on big project)
def load_metadata(metadata_file):
  try:
    with open(metadata_file, 'r') as f:
      return json.load(f)
  except FileNotFoundError:
    return {'processed_files':[], 'last_processed_index':-1}

# function to save metadata (will help track progress on big project)
def save_metadata(metadata, metadata_file):
  with open(metadata_file, 'w') as f:
    json.dump(metadata, f)

### Prep for main execution: mount Google Drive, establish file paths, check for GPU availability, and initialize OCR reader:

In [ ]:
# mount Google drive so we can access and save files with extracted text later on
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# paths
pdf_folder = "/content/drive/My Drive/polsky file split/folder_3" # this is the source of our raw PDF files
output_csv = '/content/drive/My Drive/polsky file split/export_3.csv' # this is where we'll keep our extracted text
metadata_file = '/content/drive/My Drive/polsky file split/metadata.json' # for saving progress

# check for GPU availability
if torch.cuda.is_available():
    print('GPU is available. Using GPU for OCR.')
    device = 'cuda'
else:
    print('GPU is not available. Using CPU for OCR.')
    device = 'cpu'

# initialize EasyOCR reader (only once)
reader = easyocr.Reader(['en'], gpu=True if device == 'cuda' else False)

GPU is not available. Using CPU for OCR.
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

### Main Execution: work through the entire dataset in batches

In [ ]:
# batch processing
batch_size = 20
metadata = load_metadata(metadata_file)
pdf_files = [f for f in os.listdir(pdf_folder) if f.lower().endswith('.pdf')]
start_index = metadata['last_processed_index'] + 1
total_files = len(pdf_files)

all_data = []
if os.path.exists(output_csv): # if the file exists, load its data
  all_data = pd.read_csv(output_csv).to_dict(orient='records')

for i in tqdm(range(start_index, total_files, batch_size), desc='Processing Batches'):
  batch_end = min(i + batch_size, total_files)
  batch_files = pdf_files[i:batch_end]
  batch_data = []

  for filename in tqdm(batch_files, desc=f'Processing Files {i+1}-{batch_end}', leave=False):
    # check if current filename is already present in all_data list (which is populated from the CSV if it exists)
    # if filename is found, skip the processing for that file and move to the next one.
    if filename in [d['filename'] for d in all_data]:
      continue
    pdf_path = os.path.join(pdf_folder, filename)
    text = extract_text_with_pdfplumber(pdf_path)
    ocr_text = ''

    if not text.strip() or len(text.split()) < 200: # if pdfplumber's text was < 200 words, try OCR
      ocr_text = extract_text_with_ocr(pdf_path)
      if not ocr_text:
        print(f'Could not extract text from {filename}')
      else:
        print(f'OCR used for {filename}')

    batch_data.append({'filename': filename, 'text': text, 'ocr_text': ocr_text})

  all_data.extend(batch_data)
  df = pd.DataFrame(all_data)
  df.to_csv(output_csv, index=False)

  metadata['processed_files'].extend([f['filename'] for f in batch_data])
  metadata['last_processed_index'] = batch_end -1
  save_metadata(metadata, metadata_file)

print('\nfinished processing all files')

Processing Files 1-20:   5%|▌         | 1/20 [01:59<37:54, 119.73s/it]

OCR used for ARCD.P0745US.P1 - 11_8_21 Efiled Provisional Application, ADS, EFS Ack Receipt.PDF-!!-6374506.PDF



Processing Files 1-20:  10%|█         | 2/20 [04:25<40:33, 135.17s/it]

OCR used for ARCD.P0744US.P1 - 11_09_21  Efiled Provisional Application, ADS, EFS Ack Receipt.PDF-!!-6388967.PDF



Processing Files 1-20:  15%|█▌        | 3/20 [07:28<44:27, 156.92s/it]

...

OCR used for ARCD.P0765US.P1_03_25_2022_Filing Receipt.pdf-!!-7005433.pdf



Processing Files 61-80:  60%|██████    | 12/20 [04:28<01:20, 10.02s/it]

OCR used for ARCD.P0763US.P1_02_25_2022_Filing Receipt.pdf-!!-7005753.pdf



Processing Files 61-80:  65%|██████▌   | 13/20 [04:29<00:52,  7.48s/it]

OCR used for ARCD.P0748US.P1_03_08_2022_Notice of Abandonment.pdf-!!-7005793.pdf


### Inspect text extraction results

In [ ]:
df = pd.read_csv(output_csv)

In [ ]:
# Find rows where 'text' is empty - make sure we're capturing data with OCR in those cases
empty_text_rows = df[df['text'].isnull()]
empty_text_rows

In [ ]:
# make a copy of 'df' and sort it by the length of the 'text' column (low to high)

df_copy = df.copy()
df_sorted = df_copy.sort_values(by='text', key=lambda x: x.str.len())
df_sorted['text_length'] = df_sorted['text'].apply(lambda x: len(str(x).split()))
df_sorted['ocr_text_length'] = df_sorted['ocr_text'].apply(lambda x: len(str(x).split()))

In [ ]:
df_sorted.head(20)